### calculate a time dependent proxy to R0

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline

pd.plotting.register_matplotlib_converters()

import seaborn as sns

plt.rcParams['figure.figsize'] = (12,3)

In [ ]:
#estimate each patient's  infection date, 

#estimate number of patient walk freely in the streets

#it would be helpful if city of patient and isolated date is provided

In [ ]:
#assume each patient has equal probability to be discovered after 5 days

In [ ]:
#factor to consider
#r0 decreases as people's awareness increase 
    #model by:
    #a) flat
    #b) step
#most of people have no sympton in first 5-7 days but can infect people
#time to confirm from infected is modeled by:
    #a) flat
    #b) linear till flat
    

#imported cases(and filtered at the airport) are not in the street

#assume test is instant and same day as isolation

In [ ]:
dtime = pd.read_csv('/kaggle/input/coronavirusdataset/time.csv')

dtime['date'] = pd.to_datetime(dtime['date'])

dtime['day'] = dtime['date'].dt.dayofyear

In [ ]:
dtime_pad = dtime[0:19].copy()

dtime_pad['date'] = [dtime['date'].values[0] + pd.Timedelta(days= i) for i in range(-19,0)]

for c in dtime_pad.columns[1:]:
    dtime_pad[c] = 0

dtime_pad['day'] = dtime_pad['date'].dt.dayofyear

In [ ]:
df = pd.concat([dtime_pad,dtime],axis=0)

In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
df.tail(2)

In [ ]:
df.columns

In [ ]:
df['new_confirmed'] =( df['confirmed']- df['confirmed'].shift()).fillna(0)


### main model assumption:

In [ ]:
#impossible to be caught in the first 5 days
#constant afterward
#100% caught at 15 days because he/she is very sick(?)
#not contagious in first 5 days

prob_caught = np.hstack( [[0]*5,[0.1]*10])
nday_not_contagious = 5

In [ ]:
# new prob model 2
#prob_caught = np.hstack( [[0]*10,[0.2]*5])
#nday_not_contagious = 10

In [ ]:
# new prob model 3
#prob_caught = np.hstack( [[0]*1,[1/14]*14])
#nday_not_contagious = 1

In [ ]:
plt.plot(prob_caught)
plt.title('prob caught(isolated) vs days after infected')

In [ ]:
plt.plot(np.arange(-15,0),prob_caught[::-1])
plt.title('prob density of infection day if isolated at day 0')

In [ ]:
#estimate infected date
df['infected_prob'] = 0.
for irow in range(16,len(df)):
    nnew = df['new_confirmed'].values[irow]
    for iinfected in range(15):
        df.loc[irow-15+iinfected,'infected_prob'] += nnew*(prob_caught[::-1][iinfected])

df['known_street'] = df['infected_prob'].cumsum() - df['new_confirmed'].cumsum()

In [ ]:
#check
df['infected_prob'].sum(), df['new_confirmed'].sum()

In [ ]:
# estimate the contagious number 

df['contagious_prob'] = df['infected_prob'].shift(nday_not_contagious)
df['known_contagious'] = df['known_street'].shift(nday_not_contagious)

In [ ]:
plt.plot(df['date'][20:-10],df['infected_prob'][20:-10])
plt.plot(df['date'][20:-10],df['contagious_prob'][20:-10],'--')
plt.axvline(df['date'].values[-15])

plt.title('estimated newly infected_number and when they become contagious')

In [ ]:
plt.plot(df['date'][20:-10],df['known_street'][20:-10]+1)
plt.axvline(df['date'].values[-15])
plt.yscale('log')
plt.title('known ppl infected but not isolated')

In [ ]:
plt.plot(df['date'][20:-10],df['known_contagious'][20:-10]+1)
plt.plot(df['date'][20:-10],df['infected_prob'][20:-10],linestyle='--')
plt.axvline(df['date'].values[-15])
plt.yscale('log')
plt.title('number contagious and number newly infected')

In [ ]:
#calculate newly infected number vs number of contagious ppl in the street
df['r_i'] = (df['infected_prob']/(df['known_contagious'].shift()+1e-7)).clip(0,20)

In [ ]:
plt.plot(df['date'][20:-13], df['r_i'][20:-13])
plt.axvline(df['date'].values[-15])
plt.axhline(1,linestyle = '--')
plt.title('Newly infected to contagious ratio')

### Since average number of contagious days is 10, if the ratio falls below 0.1 means it is under control

In [ ]:
#baseline model for prediction: 
#assume const new_confirmed

In [ ]:
plt.plot(df['date'].tail(20), df['new_confirmed'].tail(20))
plt.title('latest new_confirmed')
#it seems it is reasonable to assume it is constant

In [ ]:
df['est_infected'] = df['infected_prob']

#estimate infected date
for irow in range(len(df), len(df)+15):
    
    nnew =df['new_confirmed'].values[-1]
    for iinfected in range(15):
        if irow-15+iinfected < len(df):
            df.loc[irow-15+iinfected,'est_infected'] += nnew*(prob_caught[::-1][iinfected])

In [ ]:
df['est_street'] = df['est_infected'].cumsum() - df['new_confirmed'].cumsum()
df['est_contagious'] = df['est_street'].shift(nday_not_contagious)

In [ ]:
plt.plot(df['date'][20:-10],df['est_contagious'][20:-10]+1)
plt.plot(df['date'][20:-10],df['est_infected'][20:-10],linestyle='--')
plt.axvline(df['date'].values[-15])
plt.yscale('log')
plt.title('number contagious and number newly infected')

In [ ]:
#calculate newly infected number vs number of contagious ppl in the street
df['r_i_est'] = (df['est_infected']/(df['est_contagious'].shift()+1e-7)).clip(0,20)

In [ ]:
plt.plot(df['date'][40:], df['r_i_est'][40:])
plt.axvline(df['date'].values[-15])
plt.axhline(0.1,linestyle = '--')
plt.ylim(0,1)
plt.title('estimated Newly infected to contagious ratio')

In [ ]:
df['r0_proxy'] = df['r_i_est']*10

In [ ]:
#it seems like it is under control on 27-feb

df.loc[df['date'].between(pd.to_datetime('2020-02-22'),pd.to_datetime('2020-03-01')),['date','r_i_est','r0_proxy']]

In [ ]:
#latest r0_proxy estimate
df[['date','r_i_est','r0_proxy']].tail()

In [ ]:
plt.plot(df['date'][20:], df['r0_proxy'][20:])
plt.axvline(df['date'].values[-15])
plt.axhline(1,linestyle = '--')
plt.title('Time dependent R0')

In [ ]:
plt.plot(df['date'][-25:], df['r0_proxy'][-25:])
plt.axvline(df['date'].values[-15])
plt.axhline(1,linestyle = '--')
plt.title('Time dependent R0 last 25 days')

In [ ]:
#please let me know any flaw in the model. 